In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import wavfile as wav

data = np.load('../uma8_sampling/data.npy')

fs = 48000
n_channels = 7
data = data[:, :n_channels+1]

# plot each channel
plt.subplot(n_channels, 1, 1)
for i_channel in range(n_channels):
    plt.subplot(n_channels, 1, i_channel+1)
    plt.plot((1 << 15)*data[:, i_channel])
    plt.ylabel(f'{i_channel}')
plt.xlabel('Sample')
plt.show()

In [2]:
# cut out the last channel
data = data[:, :-1]

data.shape

(262144, 7)

In [3]:
# pad the data with zeroes at the end
data = np.pad(data, ((0, 1000), (0, 0)), 'constant')

data.shape

(263144, 7)

In [4]:
c = 343
X = np.array([ # estimated positions of the microphones
    [0, 0],
    [0, 0.045],
    [0.039, 0.023],
    [0.039, -0.023],
    [0, -0.045],
    [-0.039, -0.023],
    [-0.039, 0.023],
])
# target_dir = np.array([-0.615, -0.788]) # intervening direction was about this way
target_dir = np.array([1, 0]) # target was about this way (direction of USB port)

delay = (fs/c)*X@target_dir+8

list(zip(X, delay))

[(array([0., 0.]), 8.0),
 (array([0.   , 0.045]), 8.0),
 (array([0.039, 0.023]), 13.457725947521865),
 (array([ 0.039, -0.023]), 13.457725947521865),
 (array([ 0.   , -0.045]), 8.0),
 (array([-0.039, -0.023]), 2.5422740524781338),
 (array([-0.039,  0.023]), 2.5422740524781338)]

In [5]:
rounded_delay = (delay+0.5).astype(int)

list(zip(X, rounded_delay))

[(array([0., 0.]), 8),
 (array([0.   , 0.045]), 8),
 (array([0.039, 0.023]), 13),
 (array([ 0.039, -0.023]), 13),
 (array([ 0.   , -0.045]), 8),
 (array([-0.039, -0.023]), 3),
 (array([-0.039,  0.023]), 3)]

In [6]:
data.shape

(263144, 7)

In [7]:
# roll each channel
for i_channel in range(n_channels):
    data[:, i_channel] = np.roll(data[:, i_channel], rounded_delay[i_channel])

# sum each channel and normalize the result
final = np.sum(data, axis=1)
final = final/np.max(np.abs(final))

# plot the final signal
plt.plot((1 << 15)*final)
plt.show()

In [8]:
# export the final signal
wav.write('uma8_recovery_final.wav', fs, ((1 << 15)*final).astype(np.int16))

In [9]:
# export the original signals too
for i_channel in range(n_channels):
    normed_channel = data[:, i_channel]/np.max(np.abs(data[:, i_channel]))
    wav.write(f'uma8_recovery_{i_channel}.wav', fs, ((1 << 15)*normed_channel).astype(np.int16))